# FrenchRoastPy aka Transaction Parsing

## Create dataframe of needed data

In [1]:
import pandas as pd
import csv
import json

In [2]:
df = pd.read_csv('resources/secondset.csv',
                 index_col='transactionId',
                 usecols=[0, 1, 2])

In [3]:
df = df.drop_duplicates(subset=['customerId', 'accountId'])

In [4]:
df

,customerId,accountId
transactionId,,
1,14,22
2,14,21
3,10,20
4,11,24
5,14,25
6,14,26
7,12,22
8,12,26
9,12,23


## Create dataframe for tracking balances

In [5]:
# .set_index creates a multi index
# .sort_index for readability
balances = df.set_index(['customerId', 'accountId']).sort_index()

In [12]:
# add a balances column
balances['balance'] = 0.0

In [13]:
balances

balance
customerId accountId         
10         20             0.0
           22             0.0
           25             0.0
11         21             0.0
           24             0.0
12         20             0.0
           21             0.0
           22             0.0
           23             0.0
           24             0.0
           26             0.0
13         24             0.0
           25             0.0
14         21             0.0
           22             0.0
           24             0.0
           25             0.0
           26             0.0

## Calculate balances per account

In [14]:
# My initial idea of using dfs as source and target may be a bad idea. Iterating over rows in pandas is slow and not recommended.
# Next idea, read the csv directly and store results in a df. If that works I can clean up the df creation process.

In [15]:
# for each line in csv
# determine transaction type
# increment or decrement balance in appropriate account 

In [16]:
with open('resources/secondset.csv', 'r') as csv_file:
    file = csv.DictReader(csv_file)

    for row in file:
        customer_id = int(row['customerId'])
        account_id = int(row['accountId'])
        transaction_type = row['transactionType']
        amount = float(row['amount'])

        target_cell = (customer_id, account_id), ('balance')
        
        if transaction_type == 'deposit':
            balances.loc[target_cell] += amount
        if transaction_type == 'withdrawal':
            balances.loc[target_cell] -= amount

In [17]:
balances

balance
customerId accountId         
10         20         -238.24
           22          432.93
           25          583.73
11         21          812.45
           24         -542.67
12         20         -907.89
           21         -976.01
           22        -1695.83
           23         -374.90
           24         -697.57
           26         -666.25
13         24         -906.81
           25         -196.11
14         21        -1765.03
           22         -663.59
           24          530.19
           25         -394.00
           26          480.25

In [18]:
# let's convert to json and see what it looks like?
balances.to_json(r'resources/output2.json')